In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./scholarship.csv',sep=',')
df = df.dropna()

#장학종류 더미변수화
df = df.join(df["scholarship_type"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#선발대상 더미변수화
df = df.join(df["target"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())


#선발인원
df["선발 인원(명)"]=df["recruit"].str.extract(r'(\d+)')    #숫자만 남기기
df=df.replace({"선발 인원(명)":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"선발 인원(명)":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"선발 인원(명)":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"선발 인원(명)":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"선발 인원(명)":'int'})                   
student_bins=[0,1,11,51,50000]                        #범주화  0:인원미정/1-10(명):소/11-50(명):중/51-50000(명):대
student_scale=["인원미정","소","중","대"]               
df["선발인원규모"]=pd.cut(df["선발 인원(명)"],bins=student_bins,right=False,labels=student_scale) #분포범주화

#장학혜택
df["benefit"]=df["benefit"].str.replace(',',"")           #쉼표없애기(금액 때문)
df["장학혜택(만원)"]=df["benefit"].str.extract(r'(\d+)')   #숫자만남기기
df=df.replace({"장학혜택(만원)":np.NaN},0)                  #NaN(미정)값 0으로 변경                   
df=df.astype({"장학혜택(만원)":'int'})
money_bins=[0,1,101,201,50000]                               #범주화   0:금액미정/1-100(만원):소/101-200(만원):중/201-50000(만원):대
money_scale=["금액미정","소","중","대"]
df["장학혜택규모"]=pd.cut(df["장학혜택(만원)"],bins=money_bins,right=False,labels=money_scale)  #분포범주화

#열제거
df=df.drop(["scholarship_name","period","recruit","benefit"],axis='columns')

#열위치조정
df = df.reindex(columns=["idx","foundation","scholarship_type","근로장학","기숙사/주거지원","기타장학","대출지원","연수/연구장학","장학금","취업연계/산학장학","target","고등학생","기타/일반","대학생","대학원생","전문대생","중학생","청년","초등학생","선발 인원(명)","선발인원규모","장학혜택(만원)","장학혜택규모","memo"])

#csv파일저장
df=df.to_csv('scholarship_process.csv')